In [80]:
# Sources:
# https://www.youtube.com/watch?v=ujId4ipkBio
# https://medium.com/better-programming/twitter-sentiment-analysis-15d8892c0082
# https://www.youtube.com/watch?v=eFdPGpny_hY
# https://www.youtube.com/watch?v=1gQ6uG5Ujiw&t=2854s
# https://monkeylearn.com/blog/sentiment-analysis-of-twitter/

from tweepy import API 
from tweepy import Cursor
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler


from textblob import TextBlob
 
import twitter_credentials

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re


#Twitter Authenticator uses data from individual's twitter credentials 
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# TwitterClient
class TwitterClient():
    def __init__(self, twitter_user = None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client
    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets


# prints out tweets
class TwitterListener(StreamListener):

    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          




    

In [82]:
class TweetAnalyzer():

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
#sentiment analysis based of polarity

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0.5:
            return 2
        elif analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity < 0:
            return -1
        elif analysis.sentiment.polarity < -0.5:
            return -1
        else:
            return 0

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
       
        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="elonmusk", count=400)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(20))
    
    
#create a new data frame with closing price 
sentiment = df.filter(['sentiment'])

dates = df.filter(['date'])
dates_set = dates.values
#convert data frame to a Numpy array 
sentiment_set = sentiment.values

dates_set
    

                                               tweets                   id  \
0   @JoshSchoen @AlexBerenson @neil_ferguson Utter...  1257757474997690368   
1   @AlexBerenson @neil_ferguson Thanks! Something...  1257756154983153664   
2            @AlexBerenson @neil_ferguson What a tool  1257755091894800385   
3   @flcnhvy @TeslaGong @PPathole @priscillabanana...  1257559216258084864   
4   @PicklePunchD @Jason @TeslaGong @PPathole @pri...  1257554434764103680   
5   @Jason @TeslaGong @PPathole @priscillabanana T...  1257552183601422337   
6   @TeslaGong @PPathole @priscillabanana https://...  1257550522132787200   
7                      @PPathole @priscillabanana Boy  1257511340127555584   
8                      @priscillabanana X Æ A-12 Musk  1257508900812713984   
9             @Gaelic_Neilson Mom &amp; baby all good  1257466633670610944   
10  RT @SpaceX: #MayTheFourthBeWithYou https://t.c...  1257412412505862146   
11                  @Gaelic_Neilson A few hours away!  125740114

array([['2020-05-05T19:42:04.000000000'],
       ['2020-05-05T19:36:49.000000000'],
       ['2020-05-05T19:32:36.000000000'],
       ['2020-05-05T06:34:16.000000000'],
       ['2020-05-05T06:15:16.000000000'],
       ['2020-05-05T06:06:19.000000000'],
       ['2020-05-05T05:59:43.000000000'],
       ['2020-05-05T03:24:01.000000000'],
       ['2020-05-05T03:14:19.000000000'],
       ['2020-05-05T00:26:22.000000000'],
       ['2020-05-04T20:50:55.000000000'],
       ['2020-05-04T20:06:08.000000000'],
       ['2020-05-04T20:03:43.000000000'],
       ['2020-05-04T19:54:47.000000000'],
       ['2020-05-04T07:15:19.000000000'],
       ['2020-05-04T07:05:03.000000000'],
       ['2020-05-04T06:44:13.000000000'],
       ['2020-05-04T05:37:57.000000000'],
       ['2020-05-04T05:36:08.000000000'],
       ['2020-05-04T03:36:54.000000000'],
       ['2020-05-04T02:57:58.000000000'],
       ['2020-05-04T02:41:57.000000000'],
       ['2020-05-04T02:40:53.000000000'],
       ['2020-05-04T02:34:59.00000

In [83]:
import yfinance as yf

#define the ticker symbol
tickerSymbol = 'tsla'
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2020-4-1', end='2020-4-30')

tickerDf

#create a new data frame with closing price 
closing_price = tickerDf.filter(['Close'])
#convert data frame to a Numpy array 
closing_price_set = closing_price.values

closing_price_set




array([[524.  ],
       [481.56],
       [454.47],
       [480.01],
       [516.24],
       [545.45],
       [548.84],
       [573.  ],
       [650.95],
       [709.89],
       [729.83],
       [745.21],
       [753.89],
       [746.36],
       [686.72],
       [732.11],
       [705.63],
       [725.15],
       [798.75],
       [769.12],
       [800.51]])

In [71]:
# Given the array of prices, create an list of price changes. 
# An increase in stock price, compare to the previous day, will be represents with 1, 
# While a decrease in price will be represent with -1. 


price_change= [0] * (len(closing_price_set)-1)
for i in range(0, (len(closing_price_set)-1)):
    if closing_price_set[i+1] > closing_price_set[i]:
        price_change[i]= 1
    else:
        price_change[i]= -1
        
price_change

# Create an array with daily sentiment.
# Daily setiment is defined as the sum of all the tweets' sentiment score in a given day


daily_sentiment= [0] *(len(closing_price_set)-1)






[-1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1]